In [1]:
from bs4 import BeautifulSoup
import requests
import time
import re
import numpy as np
import pandas as pd

## loading auction list

In [2]:
auction_list = []

In [3]:
with open('apartments_links.txt','r') as file:
    for line in file:
        auction_list.append(line[:-1])

## collecting auction details and batch saving them

In [4]:
voivodeships = [
'dolnośląskie',
'kujawsko-pomorskie',
'łódzkie',
'lubelskie',
'lubuskie',
'małopolskie',
'mazowieckie',
'opolskie',
'podkarpackie',
'podlaskie',
'pomorskie',
'śląskie',
'świętokrzyskie',
'warmińsko-mazurskie',
'warmińsko-mazurskie',
'wielkopolskie',
'zachodniopomorskie'
]

In [5]:
start = time.time()

i = 0 # <--- change after crash
data = []

file_name = 'apartment_details.csv'

# only for first search vvvvv
f = open(file_name, "w", encoding='utf-8')
f.write('city,district,voivodeship,localization_y,localization_x,market,offer_type,area,rooms,floor,floors,build_yr,price,url\n')
f.close()
# only for first search ^^^^^

for url in auction_list[43100:43200]:
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.text, 'html.parser')
    except:
        continue

    # location
    try:
        location = soup.find_all('ul', class_="aPchu6")[0].find_all('li')

        if location[2].get_text() not in voivodeships:
            voivodeship = 'zagranica'
            city = location[3].get_text()
            district = location[2].get_text()
        else:
            voivodeship = location[2].get_text()

            if len(location)==4:
                city = location[3].get_text()
                district = ''
            elif location[3].get_text().endswith('(pow.)'):
                city = location[4].get_text().split(' ')[0]
                district = ''
            else:        
                city = location[3].get_text()
                prefixes = ['ul.','al.','Os.','pl.','rondo']
                counter = 0
                for prefix in prefixes:
                    if location[4].get_text().startswith(prefix):
                        counter += 1
                        break
                if counter > 0:
                    distirct = ''
                else:
                    district = location[4].get_text()
    except:
        voivodeship = ''
        city = ''
        district = ''

    # script for coordinates, market type, offer type
    try:
        script = soup.find('script', id='__NUXT_DATA__')
    except:
        script = []

    # coordinates
    try:
        result = re.findall(r',(\d{2}\.\d+),(\d{2}\.\d+),', script.get_text())[-1]
        localization_y = result[0]
        localization_x = result[1]
    except:
        localization_y = ''
        localization_x = ''

    # markte type
    try:
        market = re.findall(r'\\\"rynek\\\":\\\"(\w+)', script.get_text())[-1]
    except:
        market = ''  

    # offer type    
    try:
        offer_type = re.findall(r'\\\"typoferty\\\":\\\"(\w+)', script.get_text())[-1]
    except:
        offer_type = ''

    # parameters
    try:
        area = re.findall(r'\"Pow\. całkowita\",\"(\d*,?\d*)', script.get_text())[-1]
        area = area.replace(',','.')
    except:
        area = ''

    try:
        rooms = re.findall(r'\\\"number_of_rooms\\\":(\d*)', script.get_text())[-1]
    except:
        rooms = ''

    try:
        build_yr = re.findall(r'\"Rok budowy\",\"(\d{4})', script.get_text())[-1]
    except:
        build_yr = ''    

    try:
        price = re.findall(r'\\\"price\\\":(\d*)', script.get_text())[-1]
    except:
        price = 'Zapytajoce'

    try:
        floo = soup.find_all('span', class_="_2Xg-2y")[2].get_text().split()[-1]
        if '/' in floo:
            li = floo.split('/')
            floor = li[0]
            floors = li[1]
        else:
            floor = floo
            floors = ''
    except:
        floor = ''
        floors = ''

    # data collection
    data.append([city,
                district,
                voivodeship,
                localization_y,
                localization_x,
                market,
                offer_type,
                area,
                rooms,
                floor,
                floors,
                build_yr,
                price,
                url])
    
    i += 1
    if i%50 == 0 or i == len(auction_list):
        print(round(i/len(auction_list)*100,2),'%', time.strftime("%H:%M:%S", time.localtime()))
        f = open(file_name, "a", encoding='utf-8')
        for line in data:
            to_write = ','.join(line) + '\n'
            f.write(to_write)
        f.close()
        data = []
        
#     time.sleep(0.5)
        
stop = time.time()
print(round((stop-start)/60,1),'min.')

0.05 % 18:26:21
0.11 % 18:26:36
0.6 min.
